In [11]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd

In [1]:
from pathlib import Path
import polars as pl

import sys
sys.path.append("../ebnerd-benchmark/src")

from ebrec.utils._descriptive_analysis import (
    min_max_impression_time_behaviors, 
    min_max_impression_time_history
)
from ebrec.utils._polars import slice_join_dataframes
from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    truncate_history,
)
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_USER_COL
)

In [5]:
# 4 files:
# -behaviors.parquet
# -history.parquet
# -articles.parquet
# -artifacts.parquet

In [ ]:
# behaviors.parquet:
# is article id altijd een van inview articles id



# Dit laat nu de demo data, maar is handig om alles op te zetten

In [98]:
PATH = Path("../ebnerd-data-demo")
data_split = "train"

In [133]:
df_behaviors = pl.scan_parquet(PATH.joinpath(data_split, "behaviors.parquet"))
df_history = pl.scan_parquet(PATH.joinpath(data_split, "history.parquet"))
df_articles = pl.scan_parquet(PATH.joinpath("articles.parquet"))

In [134]:
behaviors = df_behaviors.collect()
history = df_history.collect()
articles = df_articles.collect()
type(history)

polars.dataframe.frame.DataFrame

In [184]:
pd_articles = articles.to_pandas()

In [203]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.28.2
    Uninstalling requests-2.28.2:
      Successfully uninstalled requests-2.28.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openxlab 0.1.0 requires packaging~=24.0, but you have packaging 23.2 which is incompatible.
openxlab 0.1.0 requires requests~=2.28.2, but you have requests 2.32.3 which is incompatible.


In [204]:
from datasets import Dataset
huggingface_data = Dataset.from_pandas(pd_articles)

In [ ]:
df_articles = pl.scan_parquet(PATH.joinpath("articles.parquet"))


In [205]:
huggingface_data.push_to_hub("testdemoarticles")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Wouter01/testdemoarticles/commit/6ef88104f95fd1ac2fd7ee7fd4a6a43690c34036', commit_message='Upload dataset', commit_description='', oid='6ef88104f95fd1ac2fd7ee7fd4a6a43690c34036', pr_url=None, pr_revision=None, pr_num=None)

In [213]:
for split in ["train", "validation"]:
    data = Dataset.from_pandas(pl.scan_parquet(PATH.joinpath(split, "behaviors.parquet")).collect().to_pandas())
    data.push_to_hub("testdemobehaviors", split=split)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/954 [00:00<?, ?B/s]

In [214]:
for split in ["train", "validation"]:
    data = Dataset.from_pandas(pl.scan_parquet(PATH.joinpath(split, "history.parquet")).collect().to_pandas())
    data.push_to_hub("testdemohistory", split=split)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/488 [00:00<?, ?B/s]

In [199]:
pd_articles.iloc[0:20]

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,ISHOCKEY: Ishockey-spilleren Sebastian Harts h...,2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ish...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/isho...,...,[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindr...",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,Hoffet tvang Prins Harry til at tage dna-test ...,2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter me...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandke...,...,"[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Da...,2023-06-29 06:21:24,False,Slingrende spritkørsel. Grove overtrædelser af...,2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn må...,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Mølle...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Erhverv, Privat virksomhed, Livsstil, Familie...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersø...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et ...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative
5,3196611,Zoo-tårnet 100 år,"I mange år var det god latin at vide, at højde...",2023-06-29 06:23:02,False,"I mange år var det god latin at vide, at højde...",2005-06-10 05:40:00,"[3067931, 3035588]",article_default,https://ekstrabladet.dk/ferie/article3196611.ece,...,[],"[Kultur, Museum og seværdighed]",539,[],ferie,NaN,NaN,NaN,0.6275,Neutral
6,3200325,Tævet ihjel på tre kvarter,Sadomasochistisk sex-guru: - Hun var en slavet...,2023-06-29 06:23:13,False,.\nSkolepige vil giftes med guru\n.\nKnus og k...,2002-06-25 05:10:00,"[3200179, 3186817]",article_default,https://ekstrabladet.dk/krimi/article3200325.ece,...,[],"[Kriminalitet, Livsstil, Personfarlig kriminal...",140,[],krimi,NaN,NaN,NaN,0.9913,Negative
7,3200913,Denne kæp kan fælde voldtægtsmand,Nye spor i den bestialske voldtægtssag i Århus,2023-06-29 06:23:15,False,"Den usædvanlig grove voldtægt, der blev begået...",2003-09-11 08:55:00,None,article_default,https://ekstrabladet.dk/krimi/article3200913.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9839,Negative
8,3209311,Morder truer med nyt drab,En morder er blevet varetægtsfængslet for at h...,2023-06-29 06:23:35,False,En morder er i retten i Kolding blevet varetæg...,2003-03-20 12:50:00,None,article_default,https://ekstrabladet.dk/krimi/article3209311.ece,...,[PER],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9975,Negative
9,3209357,Pædofil må stadig undervise børn,Lærer havde 700 børnepornobilleder på sin comp...,2023-06-29 06:23:35,False,En 56-årig sønderjysk lærer må stadig undervis...,2005-02-26 04:45:00,[3069815],article_default,https://ekstrabladet.dk/krimi/article3209357.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet, Offe...",140,[],krimi,NaN,NaN,NaN,0.7929,Negative


In [179]:
articles

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3037230,"""Ishockey-spiller: Jeg troede j…","""ISHOCKEY: Ishockey-spilleren S…",2023-06-29 06:20:57,false,"""Ambitionerne om at komme til U…",2003-08-28 08:55:00,null,"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Kriminalitet"", ""Kendt"", … ""Mindre ulykke""]",142,"[327, 334]","""sport""",null,null,null,0.9752,"""Negative"""
3044020,"""Prins Harry tvunget til dna-te…","""Hoffet tvang Prins Harry til a…",2023-06-29 06:21:16,false,"""Den britiske tabloidavis The S…",2005-06-29 08:47:00,"[3097307, 3097197, 3104927]","""article_default""","""https://ekstrabladet.dk/underh…","[""Harry"", ""James Hewitt""]","[""PER"", ""PER""]","[""Kriminalitet"", ""Kendt"", … ""Personfarlig kriminalitet""]",414,[432],"""underholdning""",null,null,null,0.7084,"""Negative"""
3057622,"""Rådden kørsel på blå plader""","""Kan ikke straffes: Udenlandske…",2023-06-29 06:21:24,false,"""Slingrende spritkørsel. Grove …",2005-10-10 07:20:00,[3047102],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Kriminalitet"", ""Transportmiddel"", ""Bil""]",118,[133],"""nyheder""",null,null,null,0.9236,"""Negative"""
3073151,"""Mærsk-arvinger i livsfare""","""FANGET I FLODBØLGEN: Skibsrede…",2023-06-29 06:21:38,false,"""To oldebørn af skibsreder Mærs…",2005-01-04 06:59:00,"[3067474, 3067478, 3153705]","""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Erhverv"", ""Privat virksomhed"", … ""Rejse""]",118,[133],"""nyheder""",null,null,null,0.9945,"""Negative"""
3193383,"""Skød svigersøn gennem babydyne""","""44-årig kvinde tiltalt for dra…",2023-06-29 06:22:57,false,"""En 44-årig mormor blev i dag f…",2003-09-15 15:30:00,null,"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9966,"""Negative"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9803492,"""Vilde billeder: Vulkan i udbru…","""Der er gang i vulkanen på Hawa…",2023-06-29 06:49:26,false,"""Det spyer med lava fra vulkane…",2023-06-08 05:49:20,"[9803493, 9803494, … 9803494]","""article_default""","""https://ekstrabladet.dk/nyhede…","[""Hawaiis"", ""Kilauea"", … ""US Geological Survey""]","[""LOC"", ""LOC"", … ""ORG""]","[""Katastrofe"", ""Vejr"", ""Større katastrofe""]",118,[133],"""nyheder""",535989,100120,4.112624e6,0.6095,"""Neutral"""
9803505,"""Flyvende Antonsen knuser topsp…","""Verdens nummer syv, Chou Tien-…",2023-06-29 06:49:26,false,"""Anders Antonsen har holdt paus…",2023-06-08 05:54:06,[9803516],"""article_default""","""https://ekstrabladet.dk/sport/…","[""Alexandra Bøje"", ""All England"", … ""Taiwan""]","[""PER"", ""PROD"", … ""LOC""]","[""Kendt"", ""Begivenhed"", … ""Sportsbegivenhed""]",142,"[327, 330]","""sport""",13320,959,55691.0,0.8884,"""Positive"""
9803525,"""Dansk skuespiller: - Jeg nægte…","""Julie R. Ølgaard fik akut kejs…",2023-06-29 06:49:26,false,"""Mens hun lå søvnløs, lød kakof…",2023-06-08 06:45:46,"[9803518, 9803519, … 9803524]","""article_default""","""https://ekstrabladet.dk/underh…","[""Cooper"", ""Englemageren"", … ""Svangerskabsforgiftning""]","[""PER"", ""PROD"", … ""MISC""]","[""Kendt"", ""Livsstil"", … ""Sygdom og behandling""]",414,[425],"""underholdning""",315391,50361,2.550671e6,0.7737,"""Negative"""


In [178]:
articles[["subtitle"]][0].to_numpy().item(), articles[["title"]][0].to_numpy().item(), articles[["premium"]][0].to_numpy().item(), articles[["body"]][0].to_numpy().item(), articles[["url"]][0].to_numpy().item()

('ISHOCKEY: Ishockey-spilleren Sebastian Harts håber stadig, at karrieren kan fortsætte i USA, efter han fik maven skåret op på diskotek',
 'Ishockey-spiller: Jeg troede jeg skulle dø',
 False,
 "Ambitionerne om at komme til USA og spille ishockey har 21-årige Sebastian Harts stadig.\nMen den tidligere Rødovre-spillere var tæt på ikke kun at miste sine sportslige ambitioner, men også livet, da han var på det forkerte sted på det forkerte tidspunkt.\nStedet var Club Mirage ved Boltens Gård i København, og tidspunktet var ved to-tiden natten til søndag 3. august.\nSammen med kæresten gennem de seneste fem år, Marie, og hendes veninde var de netop ankommet til diskoteket, da Sebastian Harts stødte ind i en person, som han ikke kendte.\n- Han gik ind i mig. Vi kom i diskussion, og jeg sagde o.k., da han foreslog, vi kunne ordne 'det' udenfor. Men vi kom på talefod igen, og han gav mig hånden og gik, husker Sebastian Harts, som i dag er hjemme og efter omstændighederne har det godt, efter d

In [110]:
history.describe()  # only use article_id_fixed

statistic,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
str,f64,f64,f64,f64,f64
"""count""",1590.0,1590.0,1590.0,1590.0,1590.0
"""null_count""",0.0,0.0,0.0,0.0,0.0
"""mean""",1.2629e6,null,null,null,null
"""std""",742510.881028,null,null,null,null
"""min""",11313.0,null,null,null,null
"""25%""",630603.0,null,null,null,null
"""50%""",1.247504e6,null,null,null,null
"""75%""",1.888545e6,null,null,null,null
"""max""",2.589391e6,null,null,null,null


In [107]:
behaviors.describe()

statistic,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",24724.0,7378.0,"""24724""",24724.0,7242.0,24724.0,24724.0,24724.0,24724.0,24724.0,1797.0,361.0,617.0,24724.0,24724.0,24051.0,21967.0
"""null_count""",0.0,17346.0,"""0""",0.0,17482.0,0.0,0.0,0.0,0.0,0.0,22927.0,24363.0,24107.0,0.0,0.0,673.0,2757.0
"""mean""",2.9276e8,9.7671e6,"""2023-05-21 17:23:50.627487""",45.315239,99.503036,1.690746,null,null,1.2489e6,0.107183,0.117974,1.027701,52.269044,0.06609,104320.226541,68.218948,69.568535
"""std""",1.6794e8,114063.51634,null,95.248077,5.397684,0.525815,null,null,741189.721112,null,0.322668,1.115202,13.60525,null,60094.28581,167.465576,32.256947
"""min""",48401.0,3.971783e6,"""2023-05-18 07:00:03""",0.0,9.0,1.0,null,null,11313.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"""25%""",1.46917363e8,9.772088e6,"""2023-05-19 19:53:02""",12.0,100.0,1.0,null,null,621402.0,null,0.0,0.0,40.0,null,52545.0,7.0,37.0
"""50%""",2.96342356e8,9.774352e6,"""2023-05-21 17:39:50""",21.0,100.0,2.0,null,null,1.246374e6,null,0.0,1.0,60.0,null,105409.0,21.0,82.0
"""75%""",4.36540299e8,9.77687e6,"""2023-05-23 11:01:42""",45.0,100.0,2.0,null,null,1.887792e6,null,0.0,2.0,60.0,null,156941.0,62.0,100.0
"""max""",5.80097289e8,9.780271e6,"""2023-05-25 06:59:52""",1799.0,100.0,3.0,null,null,2.589391e6,1.0,1.0,4.0,70.0,1.0,212376.0,1797.0,100.0


In [119]:
df_history = df_history.select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL).pipe(
    truncate_history,
    column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    history_size=10,
    padding_value=0,
    enable_warning=False,
)
df_history = df_history.collect()

In [127]:
len(df_history["user_id" == 13538]["article_id_fixed"][0])

10

In [129]:
df = slice_join_dataframes(
    df1=df_behaviors.collect(),
    df2=df_history,
    on=DEFAULT_USER_COL,
    how="left",
)
df.head(5)

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage,article_id_fixed
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32,list[i32]
48401,null,2023-05-21 21:06:50,21.0,null,2,"[9774516, 9771051, … 9759966]",[9759966],22779,false,null,null,null,false,21,16.0,27.0,"[9770333, 9769641, … 9770541]"
152513,9778745,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, … 9777397]",[9778661],150224,false,null,null,null,false,298,2.0,48.0,"[9770604, 9769622, … 9735909]"
155390,null,2023-05-24 07:30:33,45.0,null,1,"[9778369, 9777856, … 9778448]",[9777856],160892,false,null,null,null,false,401,215.0,100.0,"[9767604, 9766264, … 9770178]"
214679,null,2023-05-23 05:25:40,33.0,null,2,"[9776715, 9776406, … 9776855]",[9776566],1001055,false,null,null,null,false,1357,40.0,47.0,"[9767722, 9770882, … 9769981]"
214681,null,2023-05-23 05:31:54,21.0,null,2,"[9775202, 9776855, … 9776570]",[9776553],1001055,false,null,null,null,false,1358,5.0,49.0,"[9767722, 9770882, … 9769981]"


In [132]:
df.select(DEFAULT_CLICKED_ARTICLES_COL, DEFAULT_INVIEW_ARTICLES_COL).pipe(
    create_binary_labels_column, shuffle=True, seed=123
).with_columns(pl.col("labels").list.len().name.suffix("_len")).head(5)

article_ids_clicked,article_ids_inview,labels,labels_len
list[i32],list[i32],list[i8],u32
[9759966],"[9142581, 9774461, … 9770028]","[0, 0, … 0]",11
[9778661],"[9778728, 9777397, … 9778657]","[0, 0, … 0]",17
[9777856],"[9778155, 9777856, … 9778226]","[0, 1, … 0]",11
[9776566],"[9776497, 9776071, … 9776855]","[0, 0, … 0]",9
[9776553],"[9771995, 9776570, … 9776246]","[0, 0, … 0]",18


#### history

In [111]:
len(history[history['user_id'] == 13538]["article_id_fixed"].values[0]), len(history[history['user_id'] == 13538]["impression_time_fixed"].values[0])

ValueError: expected 5 values when selecting columns by boolean mask, got 1590

In [49]:
# history[history['user_id'] == 617963]["impression_time_fixed"].values[0] == sorted(history[history['user_id'] == 617963]["impression_time_fixed"].values[0])

In [ ]:
# okay dus die atricles zijn gesorteerd op tijd dus pak gewoon laatste 10 voor iedere user lijkt me

In [66]:
history["article_id_fixed"].shape # shape we want is 1590, 10

(1590,)

In [96]:
df_history = df_history.select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL).pipe(
    truncate_history,
    column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    history_size=10,
    padding_value=0,
    enable_warning=False,
).collect()

In [97]:
df_history[0]["article_id_fixed"][0].shape

(10,)

In [ ]:
# for every user get the 10 most recent articles with impression

In [16]:
history.describe()

,user_id
count,1.590000e+03
mean,1.262934e+06
std,7.425109e+05
min,1.131300e+04
25%,6.308670e+05
50%,1.246939e+06
75%,1.888357e+06
max,2.589391e+06


In [5]:
print(f"History: {min_max_impression_time_history(df_history).collect()}")
print(f"Behaviors: {min_max_impression_time_behaviors(df_behaviors).collect()}")

History: shape: (1, 2)
┌─────────────────────┬─────────────────────┐
│ min                 ┆ max                 │
│ ---                 ┆ ---                 │
│ datetime[μs]        ┆ datetime[μs]        │
╞═════════════════════╪═════════════════════╡
│ 2023-04-27 07:00:05 ┆ 2023-05-18 06:59:51 │
└─────────────────────┴─────────────────────┘
Behaviors: shape: (1, 2)
┌─────────────────────┬─────────────────────┐
│ min                 ┆ max                 │
│ ---                 ┆ ---                 │
│ datetime[μs]        ┆ datetime[μs]        │
╞═════════════════════╪═════════════════════╡
│ 2023-05-18 07:00:03 ┆ 2023-05-25 06:59:52 │
└─────────────────────┴─────────────────────┘


In [6]:
df_history = df_history.select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL).pipe(
    truncate_history,
    column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    history_size=30, 
    padding_value=0,
    enable_warning=False,
)
df_history.head(5).collect()

user_id,article_id_fixed
u32,list[i32]
13538,"[9767342, 9767751, … 9769366]"
58608,"[9763090, 9765545, … 9770333]"
95507,"[9768802, 9768583, … 9769450]"
106588,"[9751531, 9751633, … 9747803]"
617963,"[9765410, 9759300, … 9770798]"


In [7]:
df = slice_join_dataframes(
    df1=df_behaviors.collect(),
    df2=df_history.collect(),
    on=DEFAULT_USER_COL,
    how="left",
)
df.head(5)

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage,article_id_fixed
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32,list[i32]
48401,null,2023-05-21 21:06:50,21.0,null,2,"[9774516, 9771051, … 9759966]",[9759966],22779,false,null,null,null,false,21,16.0,27.0,"[9768820, 9767534, … 9770541]"
152513,9778745,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, … 9777397]",[9778661],150224,false,null,null,null,false,298,2.0,48.0,"[9767447, 9767739, … 9735909]"
155390,null,2023-05-24 07:30:33,45.0,null,1,"[9778369, 9777856, … 9778448]",[9777856],160892,false,null,null,null,false,401,215.0,100.0,"[9759345, 9766042, … 9770178]"
214679,null,2023-05-23 05:25:40,33.0,null,2,"[9776715, 9776406, … 9776855]",[9776566],1001055,false,null,null,null,false,1357,40.0,47.0,"[9767220, 9766886, … 9769981]"
214681,null,2023-05-23 05:31:54,21.0,null,2,"[9775202, 9776855, … 9776570]",[9776553],1001055,false,null,null,null,false,1358,5.0,49.0,"[9767220, 9766886, … 9769981]"


In [8]:
df.select(DEFAULT_CLICKED_ARTICLES_COL, DEFAULT_INVIEW_ARTICLES_COL).pipe(
    create_binary_labels_column, shuffle=True, seed=123
).with_columns(pl.col("labels").list.len().name.suffix("_len")).head(5)

article_ids_clicked,article_ids_inview,labels,labels_len
list[i32],list[i32],list[i8],u32
[9759966],"[9142581, 9774461, … 9770028]","[0, 0, … 0]",11
[9778661],"[9778728, 9777397, … 9778657]","[0, 0, … 0]",17
[9777856],"[9778155, 9777856, … 9778226]","[0, 1, … 0]",11
[9776566],"[9776497, 9776071, … 9776855]","[0, 0, … 0]",9
[9776553],"[9771995, 9776570, … 9776246]","[0, 0, … 0]",18


In [9]:
NPRATIO = 2
df.select(DEFAULT_CLICKED_ARTICLES_COL, DEFAULT_INVIEW_ARTICLES_COL).pipe(
    sampling_strategy_wu2019, npratio=NPRATIO, shuffle=False, with_replacement=True, seed=123
).pipe(create_binary_labels_column, shuffle=True, seed=123).with_columns(pl.col("labels").list.len().name.suffix("_len")).head(5)

article_ids_clicked,article_ids_inview,labels,labels_len
list[i64],list[i64],list[i8],u32
[9759966],"[9774461, 9775371, 9759966]","[0, 0, 1]",3
[9778661],"[9778661, 9777397, 9778682]","[1, 0, 0]",3
[9777856],"[9777856, 9778351, 9778448]","[1, 0, 0]",3
[9776566],"[9776566, 9776855, 9776808]","[1, 0, 0]",3
[9776553],"[9776449, 9776553, 9776570]","[0, 1, 0]",3
